In [61]:
def createlinks(start,end):
    links=[]
    for i in range(int(start),int(end)):
        links.append('https://en.wikipedia.org/wiki/{}_Mosconi_Cup'.format(i))
    return(links)

In [100]:
def createdic(start,end):
    dic={}
    for i in range(int(start),int(end)):
        dic[i]='https://en.wikipedia.org/wiki/{}_Mosconi_Cup'.format(i)
    return(dic)

In [107]:
dic = createdic(1998,2014)

https://en.wikipedia.org/wiki/1998_Mosconi_Cup
https://en.wikipedia.org/wiki/1999_Mosconi_Cup
https://en.wikipedia.org/wiki/2000_Mosconi_Cup
https://en.wikipedia.org/wiki/2001_Mosconi_Cup
https://en.wikipedia.org/wiki/2002_Mosconi_Cup
https://en.wikipedia.org/wiki/2003_Mosconi_Cup
https://en.wikipedia.org/wiki/2004_Mosconi_Cup
https://en.wikipedia.org/wiki/2005_Mosconi_Cup
https://en.wikipedia.org/wiki/2006_Mosconi_Cup
https://en.wikipedia.org/wiki/2007_Mosconi_Cup
https://en.wikipedia.org/wiki/2008_Mosconi_Cup
https://en.wikipedia.org/wiki/2009_Mosconi_Cup
https://en.wikipedia.org/wiki/2010_Mosconi_Cup
https://en.wikipedia.org/wiki/2011_Mosconi_Cup
https://en.wikipedia.org/wiki/2012_Mosconi_Cup
https://en.wikipedia.org/wiki/2013_Mosconi_Cup


In [34]:
def createtables(url):
    import requests
    from lxml import html
    page = requests.get(url)
    tree = html.fromstring(page.text)
    tables = tree.xpath('//table')
    tables=[table for table in tables if(table[0].text_content().replace('\n','')=='Results')]
    return(tables)

In [43]:
def tablelengths(tables):
    tl=[]
    for table in tables:
        tl.append(len(table)-3)
    return(tl)

In [122]:
col=['Format','European_player','European_score','American_score','American_player']
df = pd.DataFrame(columns = col)
url = dic[2013]
tables = createtables(url)
tl = tablelengths(tables)
print(tl)
fr = cdf(tables,tl,df)
fr

[5, 5, 3]


,Format,European_player,European_score,American_score,American_player
0,Teams,Team Europe,6,3,Team USA
1,Doubles,Darren Appleton Karl Boyes,6,3,Rodney Morris Earl Strickland
2,Singles,Mika Immonen,6,5,Shane Van Boening
3,Doubles,Niels Feijen Ralf Souquet,6,3,Johnny Archer Dennis Hatch
4,Singles,Darren Appleton,6,4,Rodney Morris
5,Doubles,Karl Boyes Niels Feijen,6,4,Johnny Archer Rodney Morris
6,Singles,Ralf Souquet,6,5,Dennis Hatch
7,Doubles,Darren Appleton Mika Immonen,5,6,Earl Strickland Shane Van Boening
8,Singles,Niels Feijen,6,0,Johnny Archer
9,Doubles,Mika Immonen Karl Boyes,5,6,Rodney Morris Shane Van Boening


In [121]:
def cdf(tables,tl,df):
    import pandas as pd
    import re
    l=[]
    col=['Format','European_player','European_score','American_score','American_player']

    for i,table in enumerate(tables):
        for j in range(tl[i]+1):
            try:
                element=tables[i][j].text_content().replace("\n", " ")
                match = re.match(r"\s?(\w+)\s(.+)\s([0-9])[–]([0-9])\s?\w+\s?(.+)\s", element)
                if match:
                    l.append(list(match.groups()))
            except:
                pass
    if df.empty:
        df=pd.DataFrame(l,columns=col)
    else:
        new = pd.DataFrame(l, columns = col)
        frames = (df,new)
        df = pd.concat(frames)
    return(df)

In [123]:
col=['Format','European_player','European_score','American_score','American_player']
df = pd.DataFrame(columns = col)
for link in links:
    tables = createtables(link)
    tl = tablelengths(tables)
    df = cdf(tables,tl,df)
df

,Format,European_player,European_score,American_score,American_player
0,Doubles,Mika Immonen Ralf Souquet,7,5,Johnny Archer Nick Varner
1,Doubles,Oliver Ortmann Steve Knight,7,6,James Rempe Earl Strickland
2,Doubles,Fabio Petroni Steve Davis,5,7,Kim Davenport Reed Pierce
3,Doubles,Mika Immonen Ralf Souquet,7,3,James Rempe Earl Strickland
4,Doubles,Steve Davis Fabio Petroni,3,7,Nick Varner Johnny Archer
5,Doubles,Oliver Ortmann Steve Knight,7,5,Kim Davenport Reed Pierce
6,Singles,Oliver Ortmann,3,7,Johnny Archer
7,Singles,Steve Knight,3,7,Earl Strickland
8,Singles,Mika Immonen,4,7,Reed Pierce
9,Singles,Ralf Souquet,7,1,Nick Varner


In [75]:
def main():
    import pandas as pd
    df = pd.DataFrame()
    links = createlinks('1998','2014')
    for url in links:
        tables = createtables(url)
        tl = tablelengths(tables)
        newdf = cdf(tables,tl) 
        df=df.append(newdf)
    return(df)

In [487]:
def createdf(tables,tl):
    import re
    l=[]
    col=['Format','European_player','European_score','American_score','American_player']

    for i in range(len(tables)+1):
        for j in range(1,6):
            try:
                element=tables[i][j].text_content().replace("\n", " ")
                match = re.match(r"\s?(\w+)\s(.+)\s([0-9])[–]([0-9])\s?\w+\s?(.+)\s", element)
                if match:
                    l.append(list(match.groups()))
            except:
                pass
    df=pd.DataFrame(l,columns=col)
    return(df)

In [664]:
def main():
    from lxml import html
    import requests
    import pandas as pd
    df = pd.DataFrame()
    links = createlinks('1998','2014')
    for url in links:
        tables = createtables(url)
        newdf = createdf(tables) 
        df=df.append(newdf)
    return(df)

In [710]:
pkl=r'C:\Users\andrew\Desktop\mosconi\mosconi_1998_2015_pkls'
df.to_pickle(pkl)

Missing 2014 data. Can get it from: http://www.matchroompool.com/mosconi-cup/mosconi-cup-xxi/#fixtures
compare stats here:
http://forums.azbilliards.com/showthread.php?t=439775&page=4

In [569]:
import re
l=[]
with open('manual_scrape_cleaned_2014.txt','r') as rf:
    for line in rf:
        match = re.match(r"(\w+)\s(.+)\s([0-9])\s?[-]\s?([0-9])\s?(.+)", line)
        if match:
            l.append(list(match.groups()))

In [583]:
df=pd.read_pickle(pkl)

In [670]:
df2=pd.DataFrame(l,columns=col)

In [671]:
frames=[df,df2]
df=pd.concat(frames)

In [672]:
df['Europe_won']=df.European_score>df.American_score

In [674]:
pkl=r'C:\Users\andrew\Desktop\mosconi\mosconi_1998_2014_inc_pkls'
df.to_pickle(pkl)

In [2]:
cols= ['Format','European_player','European_score','American_score','American_player','Europe_won']
df=df[cols]

NameError: name 'df' is not defined

2015 Data is backwards, need to fix the order (european player should be first)

In [705]:
def createdf_bw(tables):
    import re
    l=[]
    col=['Format','European_player','European_score','American_score','American_player']

    for i in range(len(tables)+1):
        for j in range(1,6):
            try:
                element=tables[i][j].text_content().replace("\n", " ")
                match = re.match(r"\s?(\w+)\s(.+)\s([0-9])[–]([0-9])\s?\w+\s?(.+)\s", element)
                if match:
                    print('ok')
                    lst=[match.group(1), match.group(5), match.group(4), match.group(3), match.group(2)]
                    print(lst)
                    l.append(lst)
                else:
                    print('fail')
            except:
                pass
    print(l)
    df=pd.DataFrame(l,columns=col)
    return(df)

In [701]:
def main_bw():
    from lxml import html
    import requests
    import pandas as pd
    df = pd.DataFrame()
    links = createlinks('2015','2016')
    for url in links:
        tables = createtables(url)
        newdf = createdf_bw(tables) 
        df=df.append(newdf)
    return(df)

In [706]:
df2015= main_bw()

ok
['Teams', 'Team Europe', '4', '5', 'Team USA']
ok
['Doubles', 'Karl Boyes Nick van den Berg', '5', '4', 'Corey Deuel Mike Dechaine']
ok
['Singles', 'Niels Feijen', '5', '2', 'Justin Bergman']
ok
['Doubles', 'Darren Appleton Albin Ouschan', '1', '5', 'Shane Van Boening Skyler Woodward']
ok
['Singles', 'Darren Appleton', '5', '1', 'Shane Van Boening']
ok
['Doubles', 'Niels Feijen Nick van den Berg', '3', '5', 'Justin Bergman Skyler Woodward']
ok
['Singles', 'Albin Ouschan', '5', '3', 'Mike Dechaine']
ok
['Doubles', 'Darren Appleton Karl Boyes', '5', '2', 'Shane Van Boening Corey Deuel']
ok
['Singles', 'Karl Boyes', '5', '3', 'Skyler Woodward']
ok
['Doubles', 'Darren Appleton Nick van den Berg', '4', '5', 'Shane Van Boening Mike Dechaine']
ok
['Doubles', 'Niels Feijen Albin Ouschan', '5', '1', 'Justin Bergman Corey Deuel']
ok
['Singles', 'Nick van den Berg', '4', '5', 'Corey Deuel']
ok
['Doubles', 'Karl Boyes Albin Ouschan', '5', '2', 'Mike Dechaine Skyler Woodward']
ok
['Singles', 'Ni

In [708]:
frames=[df,df2015]
df=pd.concat(frames)

In [665]:
df=main()

Next steps: Rescrape the data. 

1. Need to include the year as a multiindex and concatenate the frames per year.
1. Need to reconfigure the number of rows to possibly extend beyond 5. Have to dynamically learn the number of rows in the    given table and then use that number instead of hard coding it.
3. Need to collect the 'team' data each year. I.e. what players are on each team so that I can do analysis on the 'team' matches and include that in the final presentations.
4. Scrape location or just see if it changes every year

1. Reconfigure number of rows dynamically.